# 🔥 Curvality Fusion Validation Notebook
This notebook validates the collapse-driven Curvality fusion model using experimental D–T cross-section data.
- Compares Bayesian fits of Curvality vs standard quantum tunneling
- Derives from unified Lagrangian
- Outputs parameter posteriors and plot comparisons

In [ ]:
# 📦 Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc as pm
import arviz as az

# 📥 Upload the CSV file
from google.colab import files
uploaded = files.upload()
df = pd.read_csv('DT_Fusion_Cross_Section.csv')
energy = df['Energy (keV)'].values
xs = df['Cross Section (barns)'].values

In [ ]:
# 🔁 Curvality Bayesian Model
with pm.Model() as model_c: 
    eta0 = pm.Normal('eta0', mu=1, sigma=5)
    Tc = pm.HalfNormal('Tc', sigma=100)
    sigma = pm.HalfNormal('sigma', sigma=2)
    
    mu = eta0 * (1 - pm.math.exp(-energy / Tc))
    Y_obs = pm.Normal('Y_obs', mu=mu, sigma=sigma, observed=xs)
    trace_c = pm.sample(1000, tune=1000, chains=2, target_accept=0.95)

In [ ]:
# ⏬ Posterior Summary
az.plot_trace(trace_c)
plt.show()
az.summary(trace_c, round_to=3)

In [ ]:
# 📈 Plot Fit
eta0_m = trace_c.posterior['eta0'].mean().values
Tc_m = trace_c.posterior['Tc'].mean().values

E_plot = np.linspace(min(energy), max(energy), 300)
model_c_fit = eta0_m * (1 - np.exp(-E_plot / Tc_m))

plt.figure(figsize=(10,6))
plt.scatter(energy, xs, color='black', label='Experimental Data')
plt.plot(E_plot, model_c_fit, color='red', label='Curvality Model Fit')
plt.xlabel('Energy (keV)')
plt.ylabel('Cross Section (barns)')
plt.title('Curvality vs Experimental Fusion Data')
plt.yscale('log')
plt.grid(True, linestyle=':')
plt.legend()
plt.tight_layout()
plt.savefig('curvality_fusion_fit.png')
plt.show()

In [ ]:
# 💾 Download Output
import pickle
with open('Curvality_Fusion_Trace.pkl', 'wb') as f:
    pickle.dump(trace_c, f)

files.download('curvality_fusion_fit.png')
files.download('Curvality_Fusion_Trace.pkl')